In [ ]:
import requests
import time
import pandas as pd
import re
from datetime import date, timedelta
import json
import io

In [ ]:
!gdown 'https://drive.google.com/uc?id=****************'

In [ ]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('/content/holodilnik-bq-376311-50cc6f8cfc32.json')

In [ ]:
date_since = (date.today()-timedelta(days=182)).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
date_until = date.today().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
fields = 'appmetrica_device_id,tracker_name,install_datetime,device_type,os_name,publisher_id,publisher_name,is_reattribution,install_receive_datetime'
application_id = '4*****3'
url= f'https://api.appmetrica.yandex.ru/logs/v1/export/installations.json?application_id={application_id}&date_since={date_since}&date_until={date_until}&fields={fields}'
header = {'Authorization': 'OAuth ********************************'}
response1 = requests.get(url=url, headers=header)
k=0
while response1.status_code != 200 and k<90:
  response1 = requests.get(url=url, headers=header)
  print(response1.status_code)
  time.sleep(5)
  k+=1

In [ ]:
js1_df = response1.json()
df1 = pd.DataFrame(js1_df['data'])

In [ ]:
df_installs_rec = df1.query("is_reattribution=='false'")

In [ ]:
df_installs_rec.to_gbq('holodilnik-bq-376311.appmetrica_dashboard.installs', 
          project_id='holodilnik-bq-376311',  
          if_exists='append',
          credentials=credentials)

100%|██████████| 1/1 [00:00<00:00, 5777.28it/s]


In [ ]:
sql = """SELECT * FROM `holodilnik-bq-376311.appmetrica_dashboard.installs`"""
all_data = pd.read_gbq(sql, credentials=credentials, dialect='standard')
all_data = all_data.drop_duplicates()

In [ ]:
all_data.to_gbq('holodilnik-bq-376311.appmetrica_dashboard.installs', 
           project_id='holodilnik-bq-376311',  
           if_exists='replace',
           credentials=credentials)

100%|██████████| 1/1 [00:00<00:00, 854.76it/s]
